# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [6]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [8]:
spaceship = spaceship.dropna(how='any', axis=0)
spaceship["Cabin"]
def extract_first_part(value):
    return value.split('/')[0]

spaceship['Cabin'] = spaceship['Cabin'].apply(extract_first_part)
spaceship = spaceship.drop(["PassengerId", "Name"], axis=1)
spaceship.select_dtypes("object")
spaceship = pd.get_dummies(spaceship, columns=["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"])

target = spaceship["Transported"]
features = spaceship.drop(columns = ["Transported"])
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state = 0)

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
normalizer = RobustScaler()
#normalizer = StandardScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [9]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth = 10),
                                                     n_estimators = 100,
                                                     max_samples = 1000,
                                                     bootstrap=True)
bagging_reg.fit(X_train_norm, y_train)
bagging_reg.fit(X_train_norm,y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=10),
                 max_samples=1000, n_estimators=100)

- Evaluate your model

In [10]:
pred=bagging_reg.predict(X_test_norm)
print("R Squared", bagging_reg.score(X_test_norm, y_test))
print("Mean Squared Error", np.sqrt(mean_squared_error(y_test,pred)))

R Squared 0.43238643430303647
Mean Squared Error 0.37670066554791337


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [11]:
grid = {"n_estimators": [50,500],
        "estimator__max_leaf_nodes": [250, 1000],
        "estimator__max_depth":[10,50]}

- Run Grid Search

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
model = BaggingRegressor(DecisionTreeRegressor())
model_grid = GridSearchCV(estimator = model, param_grid = grid, cv=5)
model_grid.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 50],
                         'estimator__max_leaf_nodes': [250, 1000],
                         'n_estimators': [50, 500]})

In [16]:
model_grid.best_params_
best_model = model_grid.best_estimator_
best_model.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=50,
                                                 max_leaf_nodes=250),
                 n_estimators=500)

- Evaluate your model

In [18]:
pred=best_model.predict(X_test_norm)
print("R Squared", best_model.score(X_test_norm, y_test))
print("Mean Squared Error", np.sqrt(mean_squared_error(y_test,pred)))

R Squared 0.43021025490333165
Mean Squared Error 0.37742209298631035
